# Cоздание моделей - дефекты стали

In [1]:
import numpy as np
import pandas as pd

In [3]:
num_data = pd.read_csv("data/data.csv")

In [4]:
target = "Class_b\'2\'"
pos = num_data[target].mean()
neg = 1 - pos

In [5]:
m = num_data.size / 2
weight_for_0 = (1 / neg) 
weight_for_1 = (1 / pos) 

class_weight = {0 : weight_for_0, 1 : weight_for_1}

print(weight_for_0, weight_for_1)

1.5307570977917981 2.884101040118871


## Поделить на train, test

In [6]:
!pip install keras_preprocessing 


[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: C:\Users\kanad\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras import models
from keras import layers
from keras import optimizers 
from keras import metrics

In [8]:
def stratify_sample(data, cut, target):
    assert(cut) >= 0
    return data.groupby(target, group_keys=False).apply(lambda x: x.sample(frac=1/cut))

In [9]:
def model_train(model, split, epochs, random_target=False):
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              ) 
    
    x_train, x_test, y_train, y_test = split

    if random_target:
        y_train = np.random.randint(2, size = len(y_train))
        y_test = np.random.randint(2, size = len(y_test))  

    history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=min(32, len(x_train)),  
                    validation_data=(x_test, y_test), 
                    class_weight=class_weight)
    return history, model

In [10]:
def sample(data, cut, target):
    assert(cut > 0)
    return data.groupby(target, group_keys=False).apply(lambda x: x.sample(frac=1/cut))    

In [11]:
def get_train_test_split(data, target, cut, random_state = None):
    num_data_new = sample(data, cut, target)
    
    train, test = train_test_split(num_data_new, test_size=0.2, random_state=random_state) 

    y_train = np.array(train[target])
    y_test = np.array(test[target])

    x_train = np.array(train.drop(columns=[target]))
    x_test = np.array(test.drop(columns=[target]))
    
    return x_train, x_test, y_train, y_test 

## Модель сети

In [12]:
params = [16, 32, 32, 1]

In [13]:
input_size = len(num_data.columns) - 1

In [14]:
input_size

33

In [15]:
def make_model(input_size, parameters):
    model = models.Sequential()

    model.add(layers.Dense(parameters[0], activation='relu', input_shape=(input_size,))) 
    for i in range(1, len(parameters)-1):
        model.add(layers.Dense(parameters[i], activation="relu"))
    model.add(layers.Dense(parameters[-1], activation='sigmoid'))
    
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              metrics=["accuracy", metrics.Precision(), metrics.Recall()]
              )
    
    return model

In [16]:
volume = len(num_data)

In [17]:
cuts = [1, 2, 4, 8, 16, 32, 64, 128, 248, 512]

In [18]:
# data_vol = [volume, 970, 647, 485, 388, 323, 277, ]
# cuts = list(map(lambda x: int(volume / x), data_vol))

In [19]:
print("Cuts are {}".format(cuts))
print("Volume of data is {}".format(list(map(lambda x: int(volume / x), cuts))))

Cuts are [1, 2, 4, 8, 16, 32, 64, 128, 248, 512]
Volume of data is [1941, 970, 485, 242, 121, 60, 30, 15, 7, 3]


In [20]:
get_model_directory = lambda series, cut: "models/{}_series/{}_cut/".format(series, cut)
get_model_name = lambda number:  "m_{}.h5".format(number)

## Логи

In [21]:
import datetime 
def log_preamble(log_file_name):
    global tries, epochs, series, file_name
    time_stamp = datetime.datetime.now()

    with open(log_file_name, "a") as log_file:
        log_file.write(f"\n\nExecuted on time is {datetime.datetime.now()}\n")
        log_file.write(f"Tries: {tries}, epochs: {epochs}, series = {series}\n")
        log_file.write(f"Network configuration is {params}\n")
        log_file.write(f"Cuts are {cuts}\n")
    return time_stamp

def log_final(log_file_name, time_stamp):
    time_stamp_new = datetime.datetime.now()
    with open(log_file_name, "a") as log_file:
        log_file.write(f"Finished successfully at {time_stamp_new}\n")
        log_file.write(f"Total time = {(time_stamp_new - time_stamp).total_seconds()}\n")

## Обучение

In [22]:
# путь к файлу модели
model_name = lambda series_num, cut_val, model_num: get_model_directory(series_num, cut_val) + get_model_name(model_num)

In [23]:
def model_train(data_split, batch_size, epochs, 
                random_target=False):
    my_model = make_model(input_size, params)
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        verbose=1,
        patience=15,
        mode='auto',
        restore_best_weights=True)

    callbacks = [early_stopping]
        
    x_train, x_test, y_train, y_test = data_split

    if random_target:
        y_train = np.random.randint(2, size = len(y_train))
        y_test = np.random.randint(2, size = len(y_test))  

    history = my_model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=min(batch_size, len(x_train)),  
                    validation_data=(x_test, y_test), 
                    callbacks = callbacks) 
    return history, my_model

In [24]:
!pip install keras==2.9.0


[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: C:\Users\kanad\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [25]:
import os 
import pickle

series = [2607] 
tries = 30 
epochs = 200
batch_size = 128 
log_file = "model_log_new.txt"
t = log_preamble(log_file)

for i, s in enumerate(series):
    for cut_number, cut in enumerate(cuts):
        split = get_train_test_split(num_data, target, cut)
        for number in range(tries):
            model_name_current = model_name(s, cut, number)
            history, model_to_save = model_train(split, batch_size, epochs) 
            
            model_to_save.save(model_name_current)
            filename = f"history/hdir_{s}/h_cut{cut_number}/num{number}"
            os.makedirs(os.path.dirname(filename), exist_ok=True) 
            with open(filename, "wb") as save_file: 
                pickle.dump(history.history, save_file)

log_final(log_file, t)

Epoch 1/200
13/13 [==============================] - 2s 43ms/step - loss: 0.7673 - accuracy: 0.3918 - precision: 0.3420 - recall: 0.8488 - val_loss: 0.6844 - val_accuracy: 0.6093 - val_precision: 0.4706 - val_recall: 0.4444
Epoch 2/200
13/13 [==============================] - 0s 5ms/step - loss: 0.6508 - accuracy: 0.6263 - precision: 0.3756 - recall: 0.1456 - val_loss: 0.6290 - val_accuracy: 0.6247 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/200
13/13 [==============================] - 0s 5ms/step - loss: 0.5951 - accuracy: 0.6585 - precision: 0.3333 - recall: 0.0019 - val_loss: 0.5928 - val_accuracy: 0.6298 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/200
13/13 [==============================] - 0s 5ms/step - loss: 0.5622 - accuracy: 0.6611 - precision: 1.0000 - recall: 0.0057 - val_loss: 0.5616 - val_accuracy: 0.6478 - val_precision: 1.0000 - val_recall: 0.0486
Epoch 5/200
13/13 [==============================] - 0s 5ms/step - loss: 0.5302 - accuracy:

## Графы

In [ ]:
# def display_loss_graph(history):
#     history_dict = history.history
#     loss_values = history_dict['loss'] 
#     val_loss_values = history_dict['val_loss']
#     epochs = range(1, len(loss_values)+1)
#     plt.plot(epochs, loss_values, 'bo', label='Training loss')
#     plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

# def display_accuracy_graph(history):
#     history_dict = history.history
#     acc = history_dict["accuracy"]
#     val_acc = history_dict['val_accuracy']
#     epochs = range(1, len(acc)+1)
#     plt.plot(epochs, acc, 'bo', label='Training accuracy')
#     plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
#     plt.title('Training and validation accuracy')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

In [ ]:
# import pickle
# import os

# # сохраняет все данные о серии
# for s in range(len(series)):
#     for i, cut in enumerate(log_cuts):
#         for x in range(tries):
#             filename = './histories/{}_series/{}_cut/m_{}'.format(s, i, x)

#             os.makedirs(os.path.dirname(filename), exist_ok=True)
#             with open(filename, "wb") as file:
#                 pickle.dump(histories[s][i][x], file)

IndexError: list index out of range

In [ ]:
# accuracy_score = [[[-1 for _ in range(tries)] for _ in log_cuts] for _ in series]

In [ ]:
# for s in range(len(series)):
#     for i, cut in enumerate(log_cuts):
#         for x in range(tries):
#             history_dict = histories[s][i][x].history["val_accuracy"]
#             accuracy_score[s][i][x] = np.max(history_dict)

In [ ]:
# plt.boxplot(accuracy_score[0])

In [ ]:
# tries = 10
# epochs = 30
# series = [0, 1]
# # histories = [[[] for cut in log_cuts] for i in series] 
# # before compiling
# initial_model = model
# log_file = open("model_log.txt", "w")
# log_file.write("Tries: {}, epochs: {}, series = {}".format(tries, epochs, series))
# for i in series:
#     for cut in log_cuts:
#         model_dir = get_model_directory(i, cut)
#         split = get_train_test_split(num_data, target, cut)
#         for number in range(tries):
#             # wipe the model every time
#             new_model = models.clone_model(initial_model)   
#             # recompile, train and save
#             history, model = model_train(new_model, split, epochs) 
            
#             model_name = model_dir + get_model_name(number)
#             models.save_model(model, filepath = model_name, include_optimizer = False)
#             # histories[i][cut].append(history)
#             # display_history(history)   

Epoch 1/30
49/49 [==============================] - 1s 5ms/step - loss: 1.4323 - val_loss: 0.7054
Epoch 2/30
49/49 [==============================] - 0s 2ms/step - loss: 1.2602 - val_loss: 0.6442
Epoch 3/30
49/49 [==============================] - 0s 3ms/step - loss: 1.1149 - val_loss: 0.5656
Epoch 4/30
49/49 [==============================] - 0s 2ms/step - loss: 0.8997 - val_loss: 0.4399
Epoch 5/30
49/49 [==============================] - 0s 2ms/step - loss: 0.6534 - val_loss: 0.3040
Epoch 6/30
49/49 [==============================] - 0s 2ms/step - loss: 0.4109 - val_loss: 0.1631
Epoch 7/30
49/49 [==============================] - 0s 3ms/step - loss: 0.2144 - val_loss: 0.0731
Epoch 8/30
49/49 [==============================] - 0s 3ms/step - loss: 0.1052 - val_loss: 0.0384
Epoch 9/30
49/49 [==============================] - 0s 2ms/step - loss: 0.0577 - val_loss: 0.0231
Epoch 10/30
49/49 [==============================] - 0s 3ms/step - loss: 0.0367 - val_loss: 0.0158
Epoch 11/30
49/49 [

In [ ]:
# import pickle
# import os

# for i, h in enumerate(histories):
#     filename = './histories/model_{}'.format(i)

#     os.makedirs(os.path.dirname(filename), exist_ok=True)
#     with open(filename, "wb") as file:
#         pickle.dump(h, file)

In [ ]:
# models.save_model(model, filepath ="./models/RANDOM.h5", include_optimizer = False)

In [ ]:
# _random_histories = []
# before compiling
# initial_model = model

# for i in range(5):
    # wipe the model every time
    # new_model = models.clone_model(initial_model)   
    # recompile, train and save
    # history = model_train_save(new_model, cuts[j], i, j, random_target=True) 
    # _random_histories.append(history)